In [2]:
import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch


os.makedirs("images", exist_ok=True)

parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.00005, help="learning rate")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--n_critic", type=int, default=5, help="number of training steps for discriminator per iter")
parser.add_argument("--clip_value", type=float, default=0.01, help="lower and upper clip value for disc. weights")
parser.add_argument("--sample_interval", type=int, default=400, help="interval betwen image samples")
opt = parser.parse_args(args=[])
print(opt)

img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
    
        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )
    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img
    

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()

# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]),
        ),
    batch_size=opt.batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.RMSprop(generator.parameters(), lr=opt.lr)
optimizer_D = torch.optim.RMSprop(discriminator.parameters(), lr=opt.lr)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

#---------
# training
#---------
batches_done = 0

for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real_imgs = Variable(imgs.type(Tensor))
        optimizer_D.zero_grad()
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))
        fake_imgs = generator(z).detach()
        loss_D = -torch.mean(discriminator(real_imgs)) + torch.mean(discriminator(fake_imgs))
        loss_D.backward()
        optimizer_D.step()
        
        for p in discriminator.parameters():
            p.data.clamp_(-opt.clip_value, opt.clip_value)
        if i % opt.n_critic == 0:
            optimizer_G.zero_grad()
            gen_imgs = generator(z)
            loss_G = -torch.mean(discriminator(gen_imgs))
            loss_G.backward()
            optimizer_G.step()
            
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, opt.n_epochs, batches_done % len(dataloader), len(dataloader), loss_D.item(), loss_G.item())
            )
            if batches_done % opt.sample_interval == 0:
                save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
            batches_done += 1

Namespace(batch_size=64, channels=1, clip_value=0.01, img_size=28, latent_dim=100, lr=5e-05, n_cpu=8, n_critic=5, n_epochs=200, sample_interval=400)
[Epoch 0/200] [Batch 0/938] [D loss: -0.010765] [G loss: 0.010670]
[Epoch 0/200] [Batch 1/938] [D loss: -0.122763] [G loss: 0.009398]
[Epoch 0/200] [Batch 2/938] [D loss: -0.397616] [G loss: 0.005747]
[Epoch 0/200] [Batch 3/938] [D loss: -0.786461] [G loss: -0.006726]
[Epoch 0/200] [Batch 4/938] [D loss: -1.328830] [G loss: -0.031797]
[Epoch 0/200] [Batch 5/938] [D loss: -1.909817] [G loss: -0.072257]
[Epoch 0/200] [Batch 6/938] [D loss: -2.517338] [G loss: -0.130372]
[Epoch 0/200] [Batch 7/938] [D loss: -3.154484] [G loss: -0.209921]
[Epoch 0/200] [Batch 8/938] [D loss: -3.778653] [G loss: -0.307757]
[Epoch 0/200] [Batch 9/938] [D loss: -4.511861] [G loss: -0.429173]
[Epoch 0/200] [Batch 10/938] [D loss: -5.293580] [G loss: -0.574866]
[Epoch 0/200] [Batch 11/938] [D loss: -5.860328] [G loss: -0.747940]
[Epoch 0/200] [Batch 12/938] [D loss

[Epoch 0/200] [Batch 117/938] [D loss: -1.244816] [G loss: -18.693834]
[Epoch 0/200] [Batch 118/938] [D loss: -1.194763] [G loss: -18.589880]
[Epoch 0/200] [Batch 119/938] [D loss: -1.740192] [G loss: -18.347321]
[Epoch 0/200] [Batch 120/938] [D loss: -1.312160] [G loss: -18.423027]
[Epoch 0/200] [Batch 121/938] [D loss: -1.097927] [G loss: -18.282061]
[Epoch 0/200] [Batch 122/938] [D loss: -1.687033] [G loss: -18.178768]
[Epoch 0/200] [Batch 123/938] [D loss: -1.534176] [G loss: -18.096506]
[Epoch 0/200] [Batch 124/938] [D loss: -1.354330] [G loss: -18.065552]
[Epoch 0/200] [Batch 125/938] [D loss: -1.512480] [G loss: -18.002480]
[Epoch 0/200] [Batch 126/938] [D loss: -0.998112] [G loss: -18.208553]
[Epoch 0/200] [Batch 127/938] [D loss: -1.369141] [G loss: -18.046654]
[Epoch 0/200] [Batch 128/938] [D loss: -1.616018] [G loss: -17.879288]
[Epoch 0/200] [Batch 129/938] [D loss: -1.464405] [G loss: -17.799454]
[Epoch 0/200] [Batch 130/938] [D loss: -1.654423] [G loss: -17.764114]
[Epoch

[Epoch 1/200] [Batch 233/938] [D loss: -0.499488] [G loss: -14.553694]
[Epoch 1/200] [Batch 234/938] [D loss: -0.304949] [G loss: -14.562753]
[Epoch 1/200] [Batch 235/938] [D loss: -0.450043] [G loss: -14.470288]
[Epoch 1/200] [Batch 236/938] [D loss: -0.620225] [G loss: -14.445906]
[Epoch 1/200] [Batch 237/938] [D loss: -0.479959] [G loss: -14.465720]
[Epoch 1/200] [Batch 238/938] [D loss: -0.230035] [G loss: -14.486274]
[Epoch 1/200] [Batch 239/938] [D loss: -0.442465] [G loss: -14.405140]
[Epoch 1/200] [Batch 240/938] [D loss: -0.268771] [G loss: -14.397415]
[Epoch 1/200] [Batch 241/938] [D loss: -0.342155] [G loss: -14.371941]
[Epoch 1/200] [Batch 242/938] [D loss: -0.429355] [G loss: -14.281201]
[Epoch 1/200] [Batch 243/938] [D loss: -0.456506] [G loss: -14.250495]
[Epoch 1/200] [Batch 244/938] [D loss: -0.375528] [G loss: -14.243963]
[Epoch 1/200] [Batch 245/938] [D loss: -0.425009] [G loss: -14.168982]
[Epoch 1/200] [Batch 246/938] [D loss: -0.341548] [G loss: -14.193735]
[Epoch

[Epoch 1/200] [Batch 351/938] [D loss: -0.209421] [G loss: -12.654213]
[Epoch 1/200] [Batch 352/938] [D loss: -0.161790] [G loss: -12.670609]
[Epoch 1/200] [Batch 353/938] [D loss: -0.189912] [G loss: -12.670194]
[Epoch 1/200] [Batch 354/938] [D loss: -0.166039] [G loss: -12.688766]
[Epoch 1/200] [Batch 355/938] [D loss: -0.134392] [G loss: -12.710400]
[Epoch 1/200] [Batch 356/938] [D loss: -0.102542] [G loss: -12.678729]
[Epoch 1/200] [Batch 357/938] [D loss: -0.161925] [G loss: -12.739648]
[Epoch 1/200] [Batch 358/938] [D loss: -0.077572] [G loss: -12.738904]
[Epoch 1/200] [Batch 359/938] [D loss: -0.135876] [G loss: -12.702680]
[Epoch 1/200] [Batch 360/938] [D loss: -0.064212] [G loss: -12.720917]
[Epoch 1/200] [Batch 361/938] [D loss: -0.201467] [G loss: -12.828794]
[Epoch 1/200] [Batch 362/938] [D loss: -0.061083] [G loss: -12.814821]
[Epoch 1/200] [Batch 363/938] [D loss: -0.137329] [G loss: -12.734127]
[Epoch 1/200] [Batch 364/938] [D loss: -0.120309] [G loss: -12.730069]
[Epoch

[Epoch 2/200] [Batch 468/938] [D loss: -0.039441] [G loss: -11.873350]
[Epoch 2/200] [Batch 469/938] [D loss: -0.078121] [G loss: -11.859013]
[Epoch 2/200] [Batch 470/938] [D loss: -0.136818] [G loss: -11.871804]
[Epoch 2/200] [Batch 471/938] [D loss: -0.061607] [G loss: -11.843859]
[Epoch 2/200] [Batch 472/938] [D loss: -0.048142] [G loss: -11.788553]
[Epoch 2/200] [Batch 473/938] [D loss: 0.036446] [G loss: -11.745369]
[Epoch 2/200] [Batch 474/938] [D loss: -0.036589] [G loss: -11.858714]
[Epoch 2/200] [Batch 475/938] [D loss: -0.016313] [G loss: -11.898796]
[Epoch 2/200] [Batch 476/938] [D loss: -0.013432] [G loss: -11.876834]
[Epoch 2/200] [Batch 477/938] [D loss: -0.068543] [G loss: -11.921165]
[Epoch 2/200] [Batch 478/938] [D loss: -0.074469] [G loss: -12.017705]
[Epoch 2/200] [Batch 479/938] [D loss: -0.070917] [G loss: -12.000713]
[Epoch 2/200] [Batch 480/938] [D loss: -0.111889] [G loss: -11.944553]
[Epoch 2/200] [Batch 481/938] [D loss: -0.049488] [G loss: -11.962298]
[Epoch 

...

[Epoch 197/200] [Batch 456/938] [D loss: -0.076813] [G loss: -2.534275]
[Epoch 197/200] [Batch 457/938] [D loss: -0.067919] [G loss: -2.482510]
[Epoch 197/200] [Batch 458/938] [D loss: -0.068253] [G loss: -2.283621]
[Epoch 197/200] [Batch 459/938] [D loss: -0.068453] [G loss: -2.450635]
[Epoch 197/200] [Batch 460/938] [D loss: -0.059669] [G loss: -2.636194]
[Epoch 197/200] [Batch 461/938] [D loss: -0.068719] [G loss: -2.493076]
[Epoch 197/200] [Batch 462/938] [D loss: -0.072295] [G loss: -2.491023]
[Epoch 197/200] [Batch 463/938] [D loss: -0.071795] [G loss: -2.512299]
[Epoch 197/200] [Batch 464/938] [D loss: -0.058409] [G loss: -2.482038]
[Epoch 197/200] [Batch 465/938] [D loss: -0.072603] [G loss: -2.525879]
[Epoch 197/200] [Batch 466/938] [D loss: -0.042000] [G loss: -2.483234]
[Epoch 197/200] [Batch 467/938] [D loss: -0.039318] [G loss: -2.398471]
[Epoch 197/200] [Batch 468/938] [D loss: -0.094836] [G loss: -2.388757]
[Epoch 197/200] [Batch 469/938] [D loss: -0.068786] [G loss: -2.

[Epoch 197/200] [Batch 573/938] [D loss: -0.083859] [G loss: -2.220677]
[Epoch 197/200] [Batch 574/938] [D loss: -0.071630] [G loss: -2.299893]
[Epoch 197/200] [Batch 575/938] [D loss: -0.058621] [G loss: -2.432633]
[Epoch 197/200] [Batch 576/938] [D loss: -0.080613] [G loss: -2.477542]
[Epoch 197/200] [Batch 577/938] [D loss: -0.052541] [G loss: -2.446722]
[Epoch 197/200] [Batch 578/938] [D loss: -0.066400] [G loss: -2.510978]
[Epoch 197/200] [Batch 579/938] [D loss: -0.069254] [G loss: -2.441558]
[Epoch 197/200] [Batch 580/938] [D loss: -0.037398] [G loss: -2.530127]
[Epoch 197/200] [Batch 581/938] [D loss: -0.066968] [G loss: -2.439227]
[Epoch 197/200] [Batch 582/938] [D loss: -0.061602] [G loss: -2.454901]
[Epoch 197/200] [Batch 583/938] [D loss: -0.066377] [G loss: -2.472484]
[Epoch 197/200] [Batch 584/938] [D loss: -0.064389] [G loss: -2.411292]
[Epoch 197/200] [Batch 585/938] [D loss: -0.094384] [G loss: -2.334779]
[Epoch 197/200] [Batch 586/938] [D loss: -0.046368] [G loss: -2.

[Epoch 198/200] [Batch 687/938] [D loss: -0.069153] [G loss: -2.273365]
[Epoch 198/200] [Batch 688/938] [D loss: -0.057881] [G loss: -2.355139]
[Epoch 198/200] [Batch 689/938] [D loss: -0.073944] [G loss: -2.357100]
[Epoch 198/200] [Batch 690/938] [D loss: -0.080868] [G loss: -2.300018]
[Epoch 198/200] [Batch 691/938] [D loss: -0.063324] [G loss: -2.466914]
[Epoch 198/200] [Batch 692/938] [D loss: -0.074472] [G loss: -2.423094]
[Epoch 198/200] [Batch 693/938] [D loss: -0.068618] [G loss: -2.351099]
[Epoch 198/200] [Batch 694/938] [D loss: -0.072500] [G loss: -2.358210]
[Epoch 198/200] [Batch 695/938] [D loss: -0.082627] [G loss: -2.243967]
[Epoch 198/200] [Batch 696/938] [D loss: -0.078503] [G loss: -2.308371]
[Epoch 198/200] [Batch 697/938] [D loss: -0.067976] [G loss: -2.234034]
[Epoch 198/200] [Batch 698/938] [D loss: -0.077037] [G loss: -2.190960]
[Epoch 198/200] [Batch 699/938] [D loss: -0.066731] [G loss: -2.203798]
[Epoch 198/200] [Batch 700/938] [D loss: -0.072566] [G loss: -2.

[Epoch 198/200] [Batch 803/938] [D loss: -0.071844] [G loss: -2.064857]
[Epoch 198/200] [Batch 804/938] [D loss: -0.061661] [G loss: -2.187956]
[Epoch 198/200] [Batch 805/938] [D loss: -0.052037] [G loss: -2.179371]
[Epoch 198/200] [Batch 806/938] [D loss: -0.054857] [G loss: -2.247484]
[Epoch 198/200] [Batch 807/938] [D loss: -0.057586] [G loss: -2.254676]
[Epoch 198/200] [Batch 808/938] [D loss: -0.063372] [G loss: -2.317441]
[Epoch 198/200] [Batch 809/938] [D loss: -0.069400] [G loss: -2.401891]
[Epoch 198/200] [Batch 810/938] [D loss: -0.083665] [G loss: -2.238107]
[Epoch 198/200] [Batch 811/938] [D loss: -0.076901] [G loss: -2.238648]
[Epoch 198/200] [Batch 812/938] [D loss: -0.058999] [G loss: -2.291859]
[Epoch 198/200] [Batch 813/938] [D loss: -0.073230] [G loss: -2.374048]
[Epoch 198/200] [Batch 814/938] [D loss: -0.054183] [G loss: -2.358170]
[Epoch 198/200] [Batch 815/938] [D loss: -0.076412] [G loss: -2.315151]
[Epoch 198/200] [Batch 816/938] [D loss: -0.077019] [G loss: -2.

[Epoch 199/200] [Batch 919/938] [D loss: -0.068222] [G loss: -2.255570]
[Epoch 199/200] [Batch 920/938] [D loss: -0.057348] [G loss: -2.100500]
[Epoch 199/200] [Batch 921/938] [D loss: -0.043114] [G loss: -2.262483]
[Epoch 199/200] [Batch 922/938] [D loss: -0.050357] [G loss: -2.165730]
[Epoch 199/200] [Batch 923/938] [D loss: -0.075412] [G loss: -2.128973]
[Epoch 199/200] [Batch 924/938] [D loss: -0.021919] [G loss: -2.216092]
[Epoch 199/200] [Batch 925/938] [D loss: -0.058833] [G loss: -2.131046]
[Epoch 199/200] [Batch 926/938] [D loss: -0.056832] [G loss: -2.154121]
[Epoch 199/200] [Batch 927/938] [D loss: -0.060128] [G loss: -2.095325]
[Epoch 199/200] [Batch 928/938] [D loss: -0.080651] [G loss: -2.074784]
[Epoch 199/200] [Batch 929/938] [D loss: -0.054305] [G loss: -2.186977]
[Epoch 199/200] [Batch 930/938] [D loss: -0.054700] [G loss: -2.148219]
[Epoch 199/200] [Batch 931/938] [D loss: -0.061454] [G loss: -2.033649]
[Epoch 199/200] [Batch 932/938] [D loss: -0.083056] [G loss: -1.